In [ ]:
from collections.abc import Iterable, Generator, Sequence
import re
from typing import IO, TypeVar, Union, Optional
from sys import stdout
import textwrap

: 

In [ ]:
_PARAGRAPH_SPLITTER = re.compile(r"\n\s*\n")


def split_paragraphs(text: str) -> list[str]:
    return _PARAGRAPH_SPLITTER.split(text)

def has_paragraphs(text: str) -> bool:
    return _PARAGRAPH_SPLITTER.search(text) is not None

has_p = """\
Hey there.

What's up?
"""

assert has_paragraphs(has_p)

no_p = """\
What up?
How you been?
"""

assert not has_paragraphs(no_p)

: 

In [ ]:
start_w_p = """

Hey ya.
"""

split_paragraphs(start_w_p)

: 

In [ ]:
class BlankLine:
    _instance = None

    def __new__(class_, *args, **kwargs):
        if not isinstance(class_._instance, class_):
            class_._instance = object.__new__(class_, *args, **kwargs)
        return class_._instance
    
    def __repr__(self) -> str:
        return "<BlankLine>"

BLANK_LINE = BlankLine()

: 

In [ ]:
T = TypeVar("T")
V = TypeVar("V")

def intersperse(
    sequence: Sequence[T],
    value: V
) -> Generator[Union[T, V], None, None]:
    for index, entry in enumerate(sequence):
        if index > 0:
            yield from value
        yield entry

list(intersperse(["a", "b", "c"], [BLANK_LINE, BLANK_LINE]))

: 

In [ ]:
_SQUISH_RE = re.compile(r'\s+')

def squish(text: str) -> str:
    return _SQUISH_RE.sub(" ", text)


spaced_out = """\
What   up dude?
How ya  been?
"""

squish(spaced_out)

: 

In [ ]:
PLACEHOLDER = "…"

class TextBlock:
    def __init__(self, src: Iterable[str], width: int):
        self._src_itr = iter(src)
        self._width = width
        self._line_num = 0
        self._col_num = 0
        self._is_done = False
        self._p_buffer = None
        self._chunk_buffer = []
    
    @property
    def is_done(self) -> bool:
        return self._is_done
    
    @property
    def width(self) -> int:
        return self._width
    
    def _print_blank(self, file: IO) -> int:
        file.write(" " * self._width)
        self._line_num += 1
        return self._width
    
    def _next_src_chunk(self) -> Optional[str]:
        src_s = next(self._src_itr, None)

        if src_s is None:
            self._is_done = True
            return None
        
        if has_paragraphs(src_s):
            self._p_buffer = intersperse(
                (squish(s) for s in split_paragraphs(src_s)),
                (BLANK_LINE, BLANK_LINE)
            )
            return next(self._p_buffer)
        
        return squish(src_s)

    def _next_chunk(self) -> Union[None, str, BlankLine]:
        if len(self._chunk_buffer) > 0:
            return self._chunk_buffer.pop(0)

        if self._p_buffer is not None:
            p_chunk = next(self._p_buffer, None)
            if p_chunk is None:
                self._p_buffer = None
            else:
                return p_chunk
        
        return self._next_src_chunk()
    
    def _end_line(self, file: IO, end: Optional[str]) -> int:
        written = self._col_num
        if end is not None:
            file.write(end)
            written += len(end)
        self._line_num += 1
        self._col_num = 0
        return written
    

    def _split_chunk(
        self,
        chunk: str,
        rem_cols: int
    ) -> tuple[list[str], list[str]]:
        tokens = textwrap.TextWrapper.wordsep_re.findall(chunk)
        index = 0
        length = 0
        while index < len(tokens) and length <= rem_cols:
            length += len(tokens[index])
            index += 1
        if length > rem_cols:
            index -= 1
        return (tokens[:index], tokens[index:])
    

    def _write_chunk(self, file: IO, chunk: str):
        file.write(chunk)
        self._col_num += len(chunk)
        if self._col_num > self._width:
            raise Exception(
                "OVERWRITE {}".format(
                    ", ".join(
                        f"{k}={v!r}" for k, v in
                        dict(
                            col_num=self._col_num,
                            chunk=chunk,
                            length=len(chunk),
                        ).items()
                    )
                )
            )

    def print_line(self, file: IO, *, end: Optional[str] = "\n") -> int:
        if self._is_done:
            # We're already done, tell the caller we didn't print shit
            return 0
        
        while True:
            chunk = self._next_chunk()

            if chunk is None:
                # We have nothing left to write.
                # 
                # `self._is_done` will have been set in `self._next_src_chunk`,
                # so we just need to tell the caller we didn't write anything.
                return self._end_line(file, end)
            
            if chunk is BLANK_LINE:
                # Just end the line
                return self._end_line(file, end)
            
            if self._col_num == 0:
                chunk = chunk.lstrip()

            chunk_length = len(chunk)
            rem_cols = self._width - self._col_num

            if chunk_length <= rem_cols:
                self._write_chunk(file, chunk)
                continue
            
            words = textwrap.TextWrapper.wordsep_re.findall(chunk)

            if len(words[0]) > rem_cols:
                if self._col_num == 0:
                    self._write_chunk(file, words.pop(0)[:-1])
                    self._write_chunk(file, PLACEHOLDER)
            else:
                while len(words[0]) + self._col_num < self._width:
                    self._write_chunk(file, words.pop(0))
            
            self._chunk_buffer = words + self._chunk_buffer

            return self._end_line(file, end)


def print_blocks(blocks: list[TextBlock], file: IO = stdout):
    while not all(b.is_done for b in blocks):
        for i, b in enumerate(blocks):
            written = 0 if b.is_done else b.print_line(file, end=None) 
            file.write(" " * (b.width - written))
            if i + 1 < len(blocks):
                file.write(" | ")
        file.write("\n")

s = """\
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Pellentesque pulvinar pellentesque habitant morbi tristique senectus et netus. Accumsan lacus vel facilisis volutpat. Nisl rhoncus mattis rhoncus urna neque viverra justo.

In egestas erat imperdiet sed euismod. Turpis cursus in hac habitasse platea dictumst quisque sagittis. In nibh mauris cursus mattis molestie a. Non consectetur a erat nam at lectus urna duis convallis. Arcu non sodales neque sodales ut etiam. Porta non pulvinar neque laoreet suspendisse interdum consectetur libero id.

Vitae sapien pellentesque habitant morbi tristique. Ac ut consequat semper viverra nam libero justo laoreet sit. Urna nunc id cursus metus aliquam eleifend mi in. Nulla posuere sollicitudin aliquam ultrices sagittis.
"""

b_1 = TextBlock(
    [s],
    width=38
)

b_2 = TextBlock(
    [s],
    width=38
)

print_blocks([b_1, b_2])

: 